<a href="https://colab.research.google.com/github/iued-uni-heidelberg/cord19/blob/main/Cord19_v02_download2text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Downloading and reading CORD19 corpus
This notebook downloads and reads the free cord19 corpus into one file. The notebook is hosted at IÜD, Heidelberg University github repository https://github.com/iued-uni-heidelberg/cord19

CORD19 (covid-19) open-source corpus is available from https://www.semanticscholar.org/cord19/download. 

Documentation is available at https://github.com/allenai/cord19

The original files are in json format. The output file is in plain text format; documents are separated (by default) by \<doc id="doc1000001"> ... \</doc> tags

The purpose of the plain text file is for further processing, e.g., generating linguistic annotation using the TreeTagger or the Standford parser for part-of-speech annotation or dependency / constituency parsing.



## Downloading CORD19 corpus

The corpus is downloaded and extracted from https://www.semanticscholar.org/cord19/download

Please check the link above: if you need the latest release of the corpus or if you would like to choose another release. Currently the 2022-06-02 release is downloaded.

File size is ~11GB (v2021-08-30)
File size is ~18GB (v2022-06-02)
expected download time ~9 min


In [1]:
# !wget https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/historical_releases/cord-19_2021-08-30.tar.gz
!wget https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/historical_releases/cord-19_2022-06-02.tar.gz

--2022-10-05 08:35:20--  https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/historical_releases/cord-19_2022-06-02.tar.gz
Resolving ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com (ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com)... 52.92.165.58
Connecting to ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com (ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com)|52.92.165.58|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18657952487 (17G) [binary/octet-stream]
Saving to: ‘cord-19_2022-06-02.tar.gz’

cord-19_2022-06-02. 100%[===================>]  17.38G  40.5MB/s    in 7m 20s  

2022-10-05 08:42:40 (40.5 MB/s) - ‘cord-19_2022-06-02.tar.gz’ saved [18657952487/18657952487]



Extracting cord-19 corpus, approximate time ~ 4 min

In [2]:
# !tar -xvzf cord-19_2021-08-30.tar.gz
!tar -xvzf cord-19_2022-06-02.tar.gz 2022-06-02/document_parses.tar.gz

2022-06-02/document_parses.tar.gz


Removing initial archive to free some disk space

In [3]:
# !rm cord-19_2021-08-30.tar.gz
!rm cord-19_2022-06-02.tar.gz
!mv 2022-06-02/document_parses.tar.gz ./document_parses.tar.gz

Removing more files to save space:

In [4]:
# removing more files to save space
# !rm --recursive 2021-08-30
# !rm --recursive document_parses/pdf_json
!rm --recursive 2022-06-02

Extracting document parsers, which contain individual articles in separate json files. This is expected to take ~ 9+ min.

In [ ]:
# !tar -tvf document_parses.tar.gz >document_parses.txt

In [ ]:
# !tar -xvzf 2021-08-30/document_parses.tar.gz
!tar -xvzf document_parses.tar.gz document_parses/pmc_json

## output:

PMC1054884.xml.json

PMC1065028.xml.json

PMC1065064.xml.json

PMC1065120.xml.json

PMC1065257.xml.json

PMC1072802.xml.json

PMC1072806.xml.json

PMC1072807.xml.json

PMC1074505.xml.json

PMC1074749.xml.json
...

~ 28G of json files


In [ ]:
# ls /content/document_parses/pmc_json >pms_json.txt

In [6]:
# !rm --recursive document_parses/pmc_json
# !rm --recursive document_parses/pdf_json
!rm document_parses.tar.gz

In [7]:
!du -sh /content/document_parses/pmc_json

28G	/content/document_parses/pmc_json


In [8]:
!mkdir /content/document_parses/pmc_json_sample

## Reading json directory and merging into text file(s)

Run this cell to create the class; then run the next cell to execute on the directory "document_parses/pmc_json"

This is a class for reading a directory with json files and writing them to a single file or split into several text file, with "split_by_docs=N", N documents in each file.

In [ ]:
# -*- coding: utf-8 -*-
# Python script to open each file, read json input and copy to one text file for subsequent processing
import os, re, sys
import json
from collections import defaultdict

class clJsonDir2txt(object):
    '''
    @author Bogdan Babych, IÜD, Heidelberg University
    @email bogdan [dot] babych [at] iued [dot] uni-heidelberg [dot] de
    a script for processing covid-19 corpus:
    @url https://www.semanticscholar.org/cord19 @url https://www.semanticscholar.org/cord19/download
        recursively reads files from a directory, and glues them together into a single corpus file

    @todo:
        working with sections - collect titles of all sections; frequent sections; select argumentative sections (e.g., discussion, analysis...)
        - to compare descriptive and argumentative parts of the corpus

        experimenting with different annotations (pos, parsing... ); MT quality evaluation...
    '''
    def __init__(self, SDirName, output_file = 'corpus_out.txt', textfilter=None, include_title = True, include_sectionNames = True, include_refs = True, include_authors = True, tag='doc', id=1000000, split_by_docs = 0, copy_docs = 0): # initialising by openning the directories
        self.SOutput_file = output_file
        self.STextFilter = textfilter
        self.RFilter = re.compile(textfilter, re.IGNORECASE | re.MULTILINE)
        self.BInclTitle = include_title # implemented
        self.BInclSectionNames = include_sectionNames # implemented
        self.BInclRefs = include_refs # not implemented yet
        self.BInclAuth = include_authors # not implemented yet
        self.STag = tag
        self.ID = id
        self.ISplitByDocs = int(split_by_docs)
        self.ICopyDocs = int(copy_docs)
        # global dictionary of section names (to check and make rules...)
        self.DSectNames = defaultdict(int)
        # print(self.ISplitByDocs)
        self.openDir(SDirName)
        self.printDictionary(self.DSectNames, 'corpus-section-names.txt')
        return


    def openDir(self, path): # implementation of recursively openning directories from a given rule directory and reading each file recursively into a string
        i = 0
        path_sample = path + '_sample'
        if self.ISplitByDocs:
            SPartFile = "part1000000" + self.SOutput_file
            FOut = open(SPartFile, 'w')
        else:
            FOut = open(self.SOutput_file, 'w')

        for root,d_names,f_names in os.walk(path):
            for f in f_names:
                i+=1
                if i%10000==0: print(str(i) + '. Processing: ' + f)
                fullpath = os.path.join(root, f)
                # print(fullpath)
                try:
                    FIn = open(fullpath,'r')
                    SIn = FIn.read()
                    # apply text filter, if not None
                    if self.STextFilter and (re.search(self.RFilter, SIn) == None): continue
                    SText2Write = self.procFile(SIn,f,i)
                    if SText2Write: FOut.write(SText2Write) # if the string is not empty then write to file
                    FIn.close()
                except:
                    print(f'file {f} cannot be read or processed')
                finally:
                    # splitting output into chunks of "split_by_docs" size
                    if self.ISplitByDocs and (i % self.ISplitByDocs == 0): # if self.ISplitByDocs == 0 then everything goes into one file; if this > 0 then
                        SPartFile = "part" + str(1000000 + i) + self.SOutput_file # generate new file name
                        FOut.flush()
                        FOut.close()
                        FOut = open(SPartFile, 'w')
                    if self.ICopyDocs and (i >= self.ICopyDocs) and (i < (self.ICopyDocs + self.ISplitByDocs)):
                        try:
                            SOutputDirN = root + '_sample'
                            SOutputFN = os.path.join(SOutputDirN, f)
                            os.system(f'cp {fullpath} {SOutputFN}')
                        except:
                            print('.')

        FOut.flush()
        FOut.close()

        return


    def procFile(self, SIn,SFNameIn,i): # sending each json string for extraction of text and attaching an correct tags to each output string output string
        STagOpen = '<' + self.STag + ' id="' + self.STag + str(self.ID + i)  + '">\n'
        STagClose = '\n</' + self.STag + '>\n\n'
        SText4Corpus = self.getJson(SIn, SFNameIn)
        if SText4Corpus:
            return STagOpen + SText4Corpus + STagClose
        else:
            print('\tNo data read from: ' + SFNameIn)
            return None


    def getJson(self, SIn, SFNameIn): # for each file-level string read from a file: managing internal structure of the covid-19 json file
        LOut = [] # collecting a list of strings
        try:
            DDoc = json.loads(SIn)
        except:
            print('\t\t' + SFNameIn + ' => error reading json2dictionary')
            return None
        # metadata:
        try:
            DMetaData = DDoc['metadata']
            if DMetaData:
                SMetaData = self.getJson_Metadata(DMetaData)
                if SMetaData: LOut.append(SMetaData)
        except:
            print('\t\t\t' + SFNameIn + ' ====> no metadata')
            DMetaData = None
        # body text
        try:
            LBodyText = DDoc['body_text']
            if LBodyText:
                SBodyText = self.getJson_BodyText(LBodyText)
                LOut.append(SBodyText)
        except:
            print('\t\t\t' + SFNameIn + ' ====> no body_text')
            LBodyText = None
        # further: to implement references

        SText = '\n\n'.join(LOut)
        return SText


    def getJson_Metadata(self, DIn): # converts interesting parts of metadata into a string
        SMetadata = ''
        LMetadata = []
        try: STitle = DIn["title"]
        except: STitle = None
        if STitle and self.BInclTitle:
            LMetadata.append(STitle)

        # to implement reading of authors' names

        if LMetadata: SMetadata = '\n\n'.join(LMetadata)
        return SMetadata


    def getJson_BodyText(self, LIn): # converts interesting parts of the body texts into a string
        SBodyText = ''
        LBodyText = []
        SSectionName0 = '' # current section name set to empty for a new text
        # todo: later, in post-processing stage for the whole corpus, maybe after lemmatization...
        # ISampleNumber = 0 # samples of 100 words in text; they do not cross section boundaries 

        for DParagraph in LIn:
            # sections added 2022-09-28
            try:
                # DParagraphs["section"] ## distinction between different sections....
                SSectionName = DParagraph["section"]
                # normalizing new section name (1)
                SSectionName = SSectionName.replace("\n", " ")

                if self.BInclSectionNames and SSectionName: # if we opted to include section names and section name is not empty
                    if SSectionName != SSectionName0: # if we found a new section name
                        # processing section name
                        SSectionName0 = SSectionName # change the current section name

                        # normalizing section name (2)
                        SSectionNameNorm = SSectionName.lower()
                        SSectionNameNorm = re.sub('[0-9\.]+', ' ', SSectionNameNorm)
                        SSectionNameNorm = re.sub('[ ]+', ' ', SSectionNameNorm)
                        SSectionNameNorm = SSectionNameNorm.strip()
                        
                        self.DSectNames[SSectionNameNorm] += 1
                        SSect4text = f'<section sName="{SSectionNameNorm}">\n{SSectionName}\n</section>'

                        LBodyText.append(SSect4text)
            except:
                print('S!',)
                continue
            # first original section (we extract text after extracting section name)
            try:
                ## DParagraphs[section] ## -- later on >> distinction between different sections....
                SParagraph = DParagraph["text"]
                LBodyText.append(SParagraph)
            except:
                print('!',)
                continue



        SBodyText = '\n\n'.join(LBodyText)
        return SBodyText

    def printDictionary(self, DFreq, SFOutDict):
        FOutDict = open(SFOutDict, 'w')
        for key, val in sorted(DFreq.items(), key=lambda x: x[1], reverse=True):
            FOutDict.write(f'{key}\t{str(val)}\n')
        FOutDict.flush()
        FOutDict.close()
    



# arguments:
'''
        sys.argv[1], # obligatory: input directory name;
            other arguments optional:
            output_file = 'covid19corpus.txt',
            textfilter = None, # if this is string, only texts containing it are collected, e.g., covid
            include_title = True, # include or exclude title
            include_refs = False, # not implemented yet: include or exclude references
            split_by_docs=0 # split by groups of n documents; if 0 then write to one file

'''

'''if __name__ == '__main__':
    OJsonDir2txt = clJsonDir2txt(sys.argv[1], output_file = 'covid19corpus.txt', textfilter=None, include_title = True, include_sectionNames = True, include_refs = False, split_by_docs=0, copy_docs=240000)
'''


## numbers from previous version
This cell will executre reading of json files into a single (or multiple) files

Change the value of "split_by_docs=0" to "split_by_docs=10000" or any number ; this will create several corpus files with 10000 or any required number fo documents per file, which you wish to have.


Approximate execution time ~10 min


File size to download ~4.3 GB

It contains ~198.000 documents,

~ 671.578.587 words

~ 19.381.647 paragraphs (including empty lines, i.e., ~10M real paragraphs)

~ 4.619.100.883 characters

## numbers for the last version
Approximate execution time ~20 min

8 BNC-size (100mw) files are generted, containg together

~ 315.000 documents

  827.118.629 words

   22.094.653 paragraphs

5.650.921.315 characters


Download time can take up to 1 hour depending on your connection speed.

To split into ~BNC size chunks (100MW), split into groups of ~40000 documents (in the following cell set "split_by_docs=20000")


In [11]:
# remove parameter textfilter='covid', to return all documents
# change parameter split_by_docs=40000 to split_by_docs=0 to return a single file instead of ~5 parts with <40000 in each
OJsonDir2txt = clJsonDir2txt("document_parses/pmc_json", output_file = 'cord19.txt', textfilter='covid', include_title = True, include_sectionNames = True, include_refs = False, split_by_docs=40000, copy_docs=240000)


10000. Processing: PMC7050215.xml.json
20000. Processing: PMC8831013.xml.json
30000. Processing: PMC9088537.xml.json
40000. Processing: PMC2779497.xml.json
50000. Processing: PMC8666100.xml.json
60000. Processing: PMC4318920.xml.json
70000. Processing: PMC8871413.xml.json
80000. Processing: PMC8798676.xml.json
90000. Processing: PMC8931271.xml.json
100000. Processing: PMC8688729.xml.json
110000. Processing: PMC7407558.xml.json
120000. Processing: PMC8686689.xml.json
130000. Processing: PMC8557104.xml.json
140000. Processing: PMC8669852.xml.json
150000. Processing: PMC6100019.xml.json
160000. Processing: PMC8117966.xml.json
170000. Processing: PMC8638798.xml.json
180000. Processing: PMC8233624.xml.json
190000. Processing: PMC7675378.xml.json
200000. Processing: PMC8196470.xml.json
210000. Processing: PMC5733625.xml.json
220000. Processing: PMC8609235.xml.json
230000. Processing: PMC7316319.xml.json
240000. Processing: PMC8491259.xml.json
250000. Processing: PMC9146581.xml.json
260000. P

writing frequent section names (ordered by descending frequency, from highest to 1)

In [12]:
!ls document_parses/pmc_json_sample | wc -l
!du -sh document_parses/pmc_json_sample

40000
3.6G	document_parses/pmc_json_sample


In [13]:
!tar cvzf document_parses_sample.tar.gz document_parses/pmc_json_sample

Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
document_parses/pmc_json_sample/PMC7561248.xml.json
document_parses/pmc_json_sample/PMC8525677.xml.json
document_parses/pmc_json_sample/PMC8963769.xml.json
document_parses/pmc_json_sample/PMC7228429.xml.json
document_parses/pmc_json_sample/PMC4091094.xml.json
document_parses/pmc_json_sample/PMC7341696.xml.json
document_parses/pmc_json_sample/PMC8160395.xml.json
document_parses/pmc_json_sample/PMC8150504.xml.json
document_parses/pmc_json_sample/PMC7476267.xml.json
document_parses/pmc_json_sample/PMC8580557.xml.json
document_parses/pmc_json_sample/PMC8828293.xml.json
document_parses/pmc_json_sample/PMC7121838.xml.json
document_parses/pmc_json_sample/PMC8620715.xml.json
document_parses/pmc_json_sample/PMC7690215.xml.json
document_parses/pmc_json_sample/PMC7141566.xml.json
document_parses/pmc_json_sample/PMC7112806.xml.json
document_parses/pmc_json_sample/PMC4221744.xml.json
document_parses/pmc_json_sample/PMC8775677.xml.js

In [ ]:
!head --lines=75 /content/corpus-section-names.txt

In [ ]:
!head --lines=1000 /content/corpus-section-names.txt > corpus-selection-names-top1000.txt

To see the number of words, paragraphs in your corpus you can use this command:

In [ ]:
# !wc covid19corpus.txt

If you have split the text into parts, you can see the number of words in each part using this command:

In [ ]:
!wc part*

   4113004  109963729  759602619 part1000000cord19.txt
   4080356  109172337  753540147 part1040000cord19.txt
   4061122  109481468  755861748 part1080000cord19.txt
   4063743  108978346  752524567 part1120000cord19.txt
   4126162  110232397  761839325 part1160000cord19.txt
   4149261  110041358  761275971 part1200000cord19.txt
   4115383  110603801  763273887 part1240000cord19.txt
   3621426   98487201  679666140 part1280000cord19.txt
  32330457  866960637 5987584404 total


In [ ]:
!gzip part1000000cord19.txt
!gzip part1040000cord19.txt
!gzip part1080000cord19.txt
!gzip part1120000cord19.txt
!gzip part1160000cord19.txt
!gzip part1200000cord19.txt
!gzip part1240000cord19.txt
!gzip part1280000cord19.txt

In [ ]:
!head --lines=500000 part1240000cord19.txt >part1240000cord19-500k.txt

selecting some examples to experiment with...

In [ ]:
!cp /content/document_parses/pmc_json/PMC9034168.xml.json PMC9034168.xml.json
!cp /content/document_parses/pmc_json/PMC7128104.xml.json PMC7128104.xml.json
!cp /content/document_parses/pmc_json/PMC8769777.xml.json PMC8769777.xml.json
!cp /content/document_parses/pmc_json/PMC7926205.xml.json PMC7926205.xml.json
!cp /content/document_parses/pmc_json/PMC8799642.xml.json PMC8799642.xml.json
!cp /content/document_parses/pmc_json/PMC7124374.xml.json PMC7124374.xml.json
!cp /content/document_parses/pmc_json/PMC8812323.xml.json PMC8812323.xml.json
!cp /content/document_parses/pmc_json/PMC7446676.xml.json PMC7446676.xml.json
!cp /content/document_parses/pmc_json/PMC7436596.xml.json PMC7436596.xml.json
!cp /content/document_parses/pmc_json/PMC8808276.xml.json PMC8808276.xml.json

In [ ]:
!wc /content/document_parses/pmc_json/PMC9034168.xml.json PMC9034168.xml.json
!wc /content/document_parses/pmc_json/PMC7128104.xml.json PMC7128104.xml.json
!wc /content/document_parses/pmc_json/PMC8769777.xml.json PMC8769777.xml.json
!wc /content/document_parses/pmc_json/PMC7926205.xml.json PMC7926205.xml.json
!wc /content/document_parses/pmc_json/PMC8799642.xml.json PMC8799642.xml.json
!wc /content/document_parses/pmc_json/PMC7124374.xml.json PMC7124374.xml.json
!wc /content/document_parses/pmc_json/PMC8812323.xml.json PMC8812323.xml.json
!wc /content/document_parses/pmc_json/PMC7446676.xml.json PMC7446676.xml.json
!wc /content/document_parses/pmc_json/PMC7436596.xml.json PMC7436596.xml.json
!wc /content/document_parses/pmc_json/PMC8808276.xml.json PMC8808276.xml.json

  2032  11275 110304 /content/document_parses/pmc_json/PMC9034168.xml.json
  2032  11275 110304 PMC9034168.xml.json
  4064  22550 220608 total
 15088  32973 482112 /content/document_parses/pmc_json/PMC7128104.xml.json
 15088  32973 482112 PMC7128104.xml.json
 30176  65946 964224 total
   913   5063  58323 /content/document_parses/pmc_json/PMC8769777.xml.json
   913   5063  58323 PMC8769777.xml.json
  1826  10126 116646 total
  6563  23308 268887 /content/document_parses/pmc_json/PMC7926205.xml.json
  6563  23308 268887 PMC7926205.xml.json
 13126  46616 537774 total
  1801  12183 109304 /content/document_parses/pmc_json/PMC8799642.xml.json
  1801  12183 109304 PMC8799642.xml.json
  3602  24366 218608 total
  9638  30388 366148 /content/document_parses/pmc_json/PMC7124374.xml.json
  9638  30388 366148 PMC7124374.xml.json
 19276  60776 732296 total
  1821  10627 102799 /content/document_parses/pmc_json/PMC8812323.xml.json
  1821  10627 102799 PMC8812323.xml.json
  3642  21254 205598 total

## working with 100MW sample (40k texts)
### selecting 100-word long samples, writing to JSon dictionary, mapping names, recording

In [ ]:
# output directory
!mkdir document_parses/pmc_json_sample02/

In [ ]:
# modifying script to select 100-word long samples
'''
algorithm:
    1. form sections as strings
    2. process sections (map names), create a record
    3. write samples in a python dictionary

    ? do we need xml output to a file ?

    architecture:
      - create list of sections from list of paragraphs
      - process each section, splitting it into samples of a pre-defined size

'''

In [ ]:
# -*- coding: utf-8 -*-
# Python script to open each file, read json input and copy to one text file for subsequent processing
import os, re, sys
import json
from collections import defaultdict

class clJsonDir2txtSamples(object):
    '''
    @author Bogdan Babych, IÜD, Heidelberg University
    @email bogdan [dot] babych [at] iued [dot] uni-heidelberg [dot] de
    a script for processing covid-19 corpus:
    @url https://www.semanticscholar.org/cord19 @url https://www.semanticscholar.org/cord19/download
        recursively reads files from a directory, and glues them together into a single corpus file

    @todo:
        working with sections - collect titles of all sections; frequent sections; select argumentative sections (e.g., discussion, analysis...)
        - to compare descriptive and argumentative parts of the corpus

        experimenting with different annotations (pos, parsing... ); MT quality evaluation...
    '''
    def __init__(self, SDirName, output_file = 'corpus_out.txt', textfilter=None, include_title = True, include_sectionNames = True, include_refs = True, include_authors = True, tag='doc', id=1000000, split_by_docs = 0, copy_docs = 0, sample_size = 0, outjsondir = 'document_parses/pmc_json_sample02'): # initialising by openning the directories
        self.SOutput_file = output_file
        self.SOutput_file_stat = 'stat_' + output_file
        self.STextFilter = textfilter
        self.RFilter = re.compile(textfilter, re.IGNORECASE | re.MULTILINE)
        self.BInclTitle = include_title # implemented
        self.BInclSectionNames = include_sectionNames # implemented
        self.BInclRefs = include_refs # not implemented yet
        self.BInclAuth = include_authors # not implemented yet
        self.STag = tag
        self.ID = id
        self.ISplitByDocs = int(split_by_docs)
        self.ICopyDocs = int(copy_docs)
        self.ISampleSize = int(sample_size)
        self.SDirJsonOutput = outjsondir

        # global dictionary of section names (to check and make rules...)
        self.DSectNames = defaultdict(int)
        # print(self.ISplitByDocs)
        self.openDir(SDirName)
        self.printDictionary(self.DSectNames, 'corpus-section-names.txt')
        return


    def openDir(self, path): # implementation of recursively openning directories from a given rule directory and reading each file recursively into a string
        i = 0
        path_sample = path + '_sample'
        if self.ISplitByDocs:
            SPartFile = "part1000000" + self.SOutput_file
            FOut = open(SPartFile, 'w')
        else:
            FOut = open(self.SOutput_file, 'w')

        for root,d_names,f_names in os.walk(path):
            for f in f_names:
                i+=1
                if i%10000==0: print(str(i) + '. Processing: ' + f)
                fullpath = os.path.join(root, f)
                ## output of Json files
                fullpathout = os.path.join(self.SDirJsonOutput, f) 
                # print(fullpath)
                try:
                    FIn = open(fullpath,'r')
                    SIn = FIn.read()
                    # apply text filter, if not None
                    if self.STextFilter and (re.search(self.RFilter, SIn) == None): continue
                    SText2Write = self.procFile(SIn,f,i)
                    if SText2Write: FOut.write(SText2Write) # if the string is not empty then write to file
                    FIn.close()
                except:
                    print(f'file {f} cannot be read or processed')
                finally:
                    # splitting output into chunks of "split_by_docs" size
                    if self.ISplitByDocs and (i % self.ISplitByDocs == 0): # if self.ISplitByDocs == 0 then everything goes into one file; if this > 0 then
                        SPartFile = "part" + str(1000000 + i) + self.SOutput_file # generate new file name
                        FOut.flush()
                        FOut.close()
                        FOut = open(SPartFile, 'w')
                    if self.ICopyDocs and (i >= self.ICopyDocs) and (i < (self.ICopyDocs + self.ISplitByDocs)):
                        try:
                            SOutputDirN = root + '_sample'
                            SOutputFN = os.path.join(SOutputDirN, f)
                            os.system(f'cp {fullpath} {SOutputFN}')
                        except:
                            print('.')

        FOut.flush()
        FOut.close()

        return


    def procFile(self, SIn,SFNameIn,i): # sending each json string for extraction of text and attaching an correct tags to each output string output string
        STagOpen = '<' + self.STag + ' id="' + self.STag + str(self.ID + i)  + '">\n'
        STagClose = '\n</' + self.STag + '>\n\n'
        DJsonOut, SText4Corpus = self.getJson(SIn, SFNameIn)
        if SText4Corpus:
            return STagOpen + SText4Corpus + STagClose
        else:
            print('\tNo data read from: ' + SFNameIn)
            return None


    def getJson(self, SIn, SFNameIn): # for each file-level string read from a file: managing internal structure of the covid-19 json file
        LOut = [] # collecting a list of strings

        LBodyTextSections = [] ## list of sections: Json body text
        DJsonOut = {}

        try:
            DDoc = json.loads(SIn)
        except:
            print('\t\t' + SFNameIn + ' => error reading json2dictionary')
            return None


        # paper id
        try:
            SPaperID = DDoc["paper_id"]
        except:
            SPaperID = ''
            print('pID!')
        DJsonOut["paper_id"] = SPaperID

        # metadata:
        try:
            DMetaData = DDoc['metadata']
            if DMetaData:
                DMetaDataOut, SMetaData = self.getJson_Metadata(DMetaData)
                if SMetaData: LOut.append(SMetaData)
        except:
            print('\t\t\t' + SFNameIn + ' ====> no metadata')
            DMetaData = None
        # body text
        try:
            LBodyText = DDoc['body_text']
            if LBodyText:
                ## implementing sampling here ~ returning list of sections first (with a paired list of section names); then sampling for sample size...

                LBodyTextSections, LBodyTextSectNames, SBodyText = self.getJson_BodyText(LBodyText) ## modified function, returns 3 arguments
                LOut.append(SBodyText)
        except:
            print('\t\t\t' + SFNameIn + ' ====> no body_text')
            LBodyText = None
        # further: to implement references

        SText = '\n\n'.join(LOut)
        DJsonOut["body_text"] = LBodyTextSections

        return DJsonOut, SText


    def getJson_Metadata(self, DIn): # converts interesting parts of metadata into a string
        SMetadata = ''
        LMetadata = []
        DMetaDataOut = {}
        try: STitle = DIn["title"]
        except: STitle = None
        if STitle and self.BInclTitle:
            LMetadata.append(STitle)

        # to implement reading of authors' names
        '''
        try:
            SPaperID = DIn["authors"]
            DMetaDataOut["authors"] = "SAuthors"
        except:
            print('au!')
        '''

        if LMetadata: SMetadata = '\n\n'.join(LMetadata)
        return DMetaDataOut, SMetadata

    ## updated function ~
    def getJson_BodyText(self, LIn): # converts interesting parts of the body texts into a string
        ## modified ~ for json output
        LBodyTextSections = [] ## output - list of sections
        LBodyTextSectionPars = [] ## element of the LBodyTextSections[] list, one section-long        
        LBodyTextSectNames = [] ##
        LBodyTextSectNamesM = [] ##


        SBodyText = '' # output (old)
        LBodyText = []
        SSectionName0 = '' # current section name set to empty for a new text
        # todo: later, in post-processing stage for the whole corpus, maybe after lemmatization...
        # ISampleNumber = 0 # samples of 100 words in text; they do not cross section boundaries 

        for DParagraph in LIn:
            # sections added 2022-09-28
            # updating and adding the section name
            try:
                # DParagraphs["section"] ## distinction between different sections....
                SSectionName = DParagraph["section"]
                # normalizing new section name (1)
                SSectionName = SSectionName.replace("\n", " ")

                if self.BInclSectionNames and SSectionName: # if we opted to include section names and section name is not empty
                    if SSectionName != SSectionName0: # if we found a new section name
                        # processing section name
                        LBodyTextSectNames.append(SSectionName0) ## record the previous section name
                        ## to implement here: call a function for mapping the section names
                        LBodyTextSectNamesM.append(SSectionName0)

                        SSectionName0 = SSectionName # change the current section name
                        ## recording previous section if not empty; then adding the name of the section to a list

                        SBodyTextSectionPars = '\n'.join(LBodyTextSectionPars)
                        LBodyTextSections.append(SBodyTextSectionPars)
                        LBodyTextSectionPars = []
                            

                        # normalizing section name (2)
                        SSectionNameNorm = SSectionName.lower()
                        SSectionNameNorm = re.sub('[0-9\.]+', ' ', SSectionNameNorm)
                        SSectionNameNorm = re.sub('[ ]+', ' ', SSectionNameNorm)
                        SSectionNameNorm = SSectionNameNorm.strip()
                        
                        self.DSectNames[SSectionNameNorm] += 1
                        SSect4text = f'<section sName="{SSectionNameNorm}">\n{SSectionName}\n</section>'

                        LBodyText.append(SSect4text)
            except:
                print('S!',)
                continue
            # first original section (we extract text after extracting section name)
            try:
                ## DParagraphs[section] ## -- later on >> distinction between different sections....
                SParagraph = DParagraph["text"]
                LBodyText.append(SParagraph)
            except:
                print('!',)
                continue



        SBodyText = '\n\n'.join(LBodyText)
        return LBodyTextSections, LBodyTextSectNamesM, SBodyText

    def printDictionary(self, DFreq, SFOutDict):
        FOutDict = open(SFOutDict, 'w')
        for key, val in sorted(DFreq.items(), key=lambda x: x[1], reverse=True):
            FOutDict.write(f'{key}\t{str(val)}\n')
        FOutDict.flush()
        FOutDict.close()
    



# arguments:
'''
        sys.argv[1], # obligatory: input directory name;
            other arguments optional:
            output_file = 'covid19corpus.txt',
            textfilter = None, # if this is string, only texts containing it are collected, e.g., covid
            include_title = True, # include or exclude title
            include_refs = False, # not implemented yet: include or exclude references
            split_by_docs=0 # split by groups of n documents; if 0 then write to one file

'''

'''if __name__ == '__main__':
    OJsonDir2txt = clJsonDir2txtSamples(sys.argv[1], output_file = 'covid19corpus.txt', textfilter=None, include_title = True, include_sectionNames = True, include_refs = False, split_by_docs=0, copy_docs=240000, sample_size = 100, outjsondir = 'document_parses/pmc_json_sample02')
'''


"if __name__ == '__main__':\n    OJsonDir2txt = clJsonDir2txt(sys.argv[1], output_file = 'covid19corpus.txt', textfilter=None, include_title = True, include_sectionNames = True, include_refs = False, split_by_docs=0, copy_docs=240000)\n"

In [ ]:
OJsonDir2txtSamples = clJsonDir2txtSamples("document_parses/pmc_json_sample", output_file = 'cord19.txt', textfilter=None, include_title = True, include_sectionNames = True, include_refs = False, split_by_docs=0, copy_docs=0, sample_size = 100, outjsondir = 'document_parses/pmc_json_sample02')
